In [1]:
import kb_interface as kbi

In [2]:
kbi.start()

Ontology successfully initialized!


In [3]:
test_create = [['downtime_kpi', 
                'mean_time_between_failures', 
                'Mean time between failures cumulative over machine-opertion pairs', 
                's', 
                'A°sum°mo[S°/[ R°bad_cycles_sum°T°m°o° ; R°time_sum°T°m°o° ]]',
                'sum_M_O(bad_cycles_sum(T,m,o)/time_sum(T,m,o))',
                True,
                True], 
               
               ['utilization_kpi', 
                'availability', 
                'Percentage of machine uptime in respect to machine downtime over each machine-operation pairs', 
                '%', 
                'S°*[ S°/[ A°sum°m[ R°time_sum°T°m°working° ] ; S°+[ A°sum°m[ R°time_sum°T°m°idle° ] ; A°sum°m[ R°time_sum°T°m°offline° ] ] ] ; C°100° ]',
                '(sum_M( time_sum(T,m,working)) / ( sum_M(time_sum(T,m,Idle)) + sum_M(time_sum(T,m,offline)) ) )*100',
                True,
                False]]

In [ ]:
for ind in kbi.ONTO.search(label='kpi')[0].instances():
    print(ind.label[0],':')
    print(kbi.get_formulas(ind.label[0][:]))

consumption_sum
(['A°sum°mo[ A°sum°t[ D°consumption_sum°t°m°o° ] ]'], ['R0MKtB1FGsxLy4i1gr1sV9'], ['consumption_sum'])
cost_sum
(['A°sum°mo[ A°sum°t[ D°cost_sum°t°m°o° ] ]'], ['R4r84xwBOFiwnSO3LWja5X'], ['cost_sum'])
consumption_max
(['A°max°mo[ A°max°t[ D°consumption_max°t°m°o° ] ]'], ['R5V6RUhYgu8fk1lTd1JfbH'], ['consumption_max'])
average_cycle_time_avg
(['A°mean°mo[ A°mean°t[ D°average_cycle_time_avg°t°m°o° ]]'], ['R7MgOWHfPwuBRiHOnNrT57l'], ['average_cycle_time_avg'])
power_sum
(['A°sum°mo[ A°sum°t[ D°power_sum°t°m°o° ] ]'], ['R7TfPiPG80rCfz98pGtetEk'], ['power_sum'])
bad_cycles_max
(['A°max°mo[ A°max°t[ D°bad_cycles_max°t°m°o° ]]'], ['R7nZyqyELJk4eJ3Lbp2PWjR'], ['bad_cycles_max'])
success_rate
(['A°sum°mo[ S°*[S°/[ R°good_cycles_sum°T°m°o° ; R°cycles_sum°T°m°o° ]] ; C°100°]', 'A°sum°mo[ A°sum°t[ D°good_cycles_sum°t°m°o° ]]', 'A°sum°mo[ A°sum°t[ D°cycles_sum°t°m°o° ]]'], ['R7u1FemtJqdMBwsG7daSEFV', 'R8LqLgWbE81e95SUOYZ4D4R', 'RB8j5Hueb5Uhq9155yIWWeE'], ['success_rate', 'good_cycle

In [5]:
for l in test_create:
    print(f"Processing: {l}")  # Mostra cosa contiene la lista l
    kbi.add_kpi(*l)

Processing: ['downtime_kpi', 'mean_time_between_failures', 'Mean time between failures cumulative over machine-opertion pairs', 's', 'A°sum°mo[S°/[ R°bad_cycles_sum°T°m°o° ; R°time_sum°T°m°o° ]]', 'sum_M_O(bad_cycles_sum(T,m,o)/time_sum(T,m,o))', True, True]
KPI LABEL ALREADY EXISTS
Processing: ['utilization_kpi', 'availability', 'Percentage of machine uptime in respect to machine downtime over each machine-operation pairs', '%', 'S°*[ S°/[ A°sum°m[ R°time_sum°T°m°working° ] ; S°+[ A°sum°m[ R°time_sum°T°m°idle° ] ; A°sum°m[ R°time_sum°T°m°offline° ] ] ] ; C°100° ]', '(sum_M( time_sum(T,m,working)) / ( sum_M(time_sum(T,m,Idle)) + sum_M(time_sum(T,m,offline)) ) )*100', True, False]
KPI LABEL ALREADY EXISTS


In [6]:
kbi.get_formulas('mean_time_between_failures')

(['A°sum°mo[S°/[ R°bad_cycles_sum°T°m°o° ; R°time_sum°T°m°o° ]]',
  'A°sum°mo[ A°sum°t[ D°bad_cycles_sum°t°m°o° ]]',
  'A°sum°mo[ A°sum°t[ D°time_sum°t°m°o° ]]'],
 ['TI7elI3KDWSoArKzluuKIS',
  'R8ScTyWAfxBTPLvxKYZkymU',
  'R8mT8WvQuTNhzof24r75lrf'],
 ['mean_time_between_failures', 'bad_cycles_sum', 'time_sum'])

In [7]:
kbi.get_formulas('availability')

(['S°*[ S°/[ A°sum°m[ R°time_sum°T°m°working° ] ; S°+[ A°sum°m[ R°time_sum°T°m°idle° ] ; A°sum°m[ R°time_sum°T°m°offline° ] ] ] ; C°100° ]',
  'A°sum°mo[ A°sum°t[ D°time_sum°t°m°o° ]]',
  'A°sum°mo[ A°sum°t[ D°time_sum°t°m°o° ]]',
  'A°sum°mo[ A°sum°t[ D°time_sum°t°m°o° ]]'],
 ['V2sWB6bQ5xYqUabIwJH44K',
  'R8mT8WvQuTNhzof24r75lrf',
  'R8mT8WvQuTNhzof24r75lrf',
  'R8mT8WvQuTNhzof24r75lrf'],
 ['availability', 'time_sum', 'time_sum', 'time_sum'])